<a href="https://colab.research.google.com/github/NoraHK3/DataSciProject/blob/main/Sayyidaty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
from bs4 import BeautifulSoup
import csv
import json
from datetime import datetime
import time
import re
from collections import OrderedDict

class SaudiDishScraper:
    def __init__(self):
        self.base_url = "https://kitchen.sayidaty.net"
        self.saudi_cuisine_url = f"{self.base_url}/recipes/index/cuisine/2419"
        self.dishes_data = []

        # 1) قاموس المكونات الأساسية (canonical -> synonyms)
        self.ING_CANON = {
            # بروتينات
            'لحم': ['لحم', 'لحمة', 'لحم غنم', 'غنم', 'ضأن', 'خروف', 'بقري', 'لحم بقر', 'بقر', 'جمل', 'لحم جمل', 'بعير'],
            'دجاج': ['دجاج', 'فراخ', 'دجاجة'],
            'سمك': ['سمك', 'سمكة', 'سي فود', 'بحري'],
            'روبيان': ['روبيان', 'جمبري', 'قريدس'],
            'بيض'  : ['بيض', 'بيضة', 'بيضات'],
            'كبد'  : ['كبد', 'كبدة', 'كبده'],

            # أساسيات/نشويات
            'رز': ['رز', 'أرز', 'ارز', 'الرز', 'رز بسمتي', 'بسمتي'],
            'قرصان': ['قرصان'],
            'جريش': ['جريش', 'جريشه'],
            'مرقوق': ['مرقوق', 'مَرْقُوق'],
            'برغل': ['برغل', 'بورغل'],
            'كسكس': ['كسكس', 'كسكسي'],
            'شعيرية': ['شعيرية', 'شعريه'],
            'مكرونة': ['مكرونة', 'معكرونة', 'مكارونه', 'معكرونه'],
            'عجين': ['عجين', 'عجينة', 'عجينه'],
            'فريكة': ['فريكة', 'فريكه'],
            'خبز': ['خبز', 'خبزه', 'خبزة', 'خبز تنور', 'خبز بر'],
            'قمح': ['قمح', 'بر', 'حنطة'],

            # بقوليات
            'عدس': ['عدس', 'عدسه', 'عدسية'],
            'حمص': ['حمص', 'حمص حب', 'حمص مطحون'],
            'فول': ['فول', 'فول مدمس', 'فول حب'],
            'بازلاء': ['بازلاء', 'بسلة', 'بازيلا'],
            'لوبيا': ['لوبيا', 'لوبية'],

            # خضروات وأعشاب
            'بطاطس': ['بطاطس', 'بطاطا', 'بطاطه'],
            'طماطم': ['طماطم', 'بندورة', 'بندوره'],
            'بصل': ['بصل', 'بصلة', 'بصلات'],
            'ثوم': ['ثوم', 'ثومه'],
            'فلفل أسود': ['فلفل أسود', 'فلفل اسود', 'فلفل'],
            'فلفل': ['فلفل', 'فلفل رومي', 'فلفل حار', 'شطة', 'شطه'],
            'جزر': ['جزر', 'جزره'],
            'كوسة': ['كوسة', 'كوسه'],
            'باذنجان': ['باذنجان'],
            'سبانخ': ['سبانخ', 'سبناخ'],
            'ملوخية': ['ملوخية', 'ملوخيا'],
            'قرع': ['قرع', 'قرعه', 'يقطين'],
            'خيار': ['خيار'],
            'ملفوف': ['ملفوف', 'كرنب'],
            'خس': ['خس', 'خص'],
            'جرجير': ['جرجير'],
            'نعناع': ['نعناع', 'نعنع'],
            'كزبرة': ['كزبرة', 'كزبره', 'كسبرة'],
            'بقدونس': ['بقدونس', 'بقدونـس'],

            # ألبان
            'لبن': ['لبن', 'لبن رائب', 'لبن عيران', 'روب'],
            'زبادي': ['زبادي', 'زبادى'],
            'حليب': ['حليب', 'حليب سائل', 'حليب بودرة'],
            'قشطة': ['قشطة', 'قشطه'],
            'جبن': ['جبن', 'جبنة', 'أجبان'],
            'سمن': ['سمن', 'سمنة', 'سمنه'],
            'زبدة': ['زبدة', 'زبد'],

            # زيوت
            'زيت زيتون': ['زيت زيتون'],
            'زيت': ['زيت', 'زيت نباتي', 'زيت ذرة', 'زيت دوار الشمس'],

            # توابل
            'ملح': ['ملح'],
            'كمون': ['كمون'],
            'كركم': ['كركم'],
            'قرفة': ['قرفة', 'دارسين'],
            'هيل': ['هيل', 'حبهان'],
            'زنجبيل': ['زنجبيل'],
            'قرنفل': ['قرنفل'],
            'زعفران': ['زعفران'],
            'ورق غار': ['ورق غار', 'ورق لورا'],
            'بهارات مشكلة': ['بهارات مشكلة', 'بهارات', 'توابل'],
            'شطة': ['شطة', 'شطه'],

            # إضافات/مكسرات
            'لوز': ['لوز', 'لوزة'],
            'جوز': ['جوز', 'عين جمل'],
            'فستق': ['فستق', 'فستق حلبي'],
            'صنوبر': ['صنوبر'],
            'سمسم': ['سمسم'],
            'زبيب': ['زبيب'],
            'تمر': ['تمر', 'تمور', 'عجوة'],
            'دبس تمر': ['دبس تمر'],
            'عسل': ['عسل', 'عسل أسود', 'عسل اسود'],
            'سكر': ['سكر', 'سكّر'],

            # سوائل/صلصات
            'خل': ['خل'],
            'ليمون': ['ليمون', 'حامض', 'عصير ليمون'],
            'ماء ورد': ['ماء ورد'],
            'ماء زهر': ['ماء زهر'],
            'مرق': ['مرق', 'مرقة', 'شوربة'],
            'معجون طماطم': ['معجون طماطم', 'صلصة طماطم', 'طماطم معجون'],

            # حلويات/أساسيات
            'سميد': ['سميد'],
            'طحين': ['طحين', 'دقيق'],
            'نشا': ['نشا', 'نشا ذرة'],
            'ماء': ['ماء', 'مويه'],
            'فانيليا': ['فانيليا', 'فانيلا'],
            'كاكاو': ['كاكاو', 'كاكاو بودرة'],
            'حليب مكثف': ['حليب مكثف'],

            # أكلات شعبية (قد تظهر كلِبِل)
            'كليجة': ['كليجة', 'كليجا'],
            'معمول': ['معمول', 'معمول تمر'],
            'بسبوسة': ['بسبوسة', 'بسبوسه'],
            'لقيمات': ['لقيمات', 'لقمة القاضي', 'عوامة'],
            'كنافة': ['كنافة', 'كنافه'],
            'قطايف': ['قطايف', 'قطائف'],
            'عصيدة': ['عصيدة', 'عصيد'],
            'بلاليط': ['بلاليط'],
            'شعيرية حلوة': ['شعيرية حلوة']
        }

        # 2) فهرس: (syn normalized) -> (canonical الأصلي كما سيُطبع)
        self.syn2canon = self._build_synonym_index(self.ING_CANON)
        # 3) ريجكس يجمع كل المرادفات (الأطول أولاً)
        self.ing_regex = self._build_regex_from_synonyms(self.syn2canon)

    # ========= أدوات نصية أساسية =========
    def normalize_ar(self, s: str) -> str:
        if not s:
            return ''
        s = re.sub(r'[\u064B-\u0652\u0670]', '', s)  # حركات
        s = s.replace('ـ', '')                        # تطويل
        s = re.sub('[إأآا]', 'ا', s)                 # الألف
        s = s.replace('ى', 'ي')                       # ألف مقصورة
        s = s.replace('ؤ', 'و').replace('ئ', 'ي')     # همزات
        s = re.sub(r'\s+', ' ', s).strip()
        return s

    def _build_synonym_index(self, canon: dict) -> dict:
        syn2canon = {}
        for canonical, variants in canon.items():
            for v in variants + [canonical]:
                v_norm = self.normalize_ar(v.replace('ة', 'ه'))  # للتطابق فقط
                syn2canon[v_norm] = canonical                    # نُرجِع النص الأصلي الجميل
        return syn2canon

    def _build_regex_from_synonyms(self, syn2canon: dict):
        alts = sorted(syn2canon.keys(), key=len, reverse=True)
        # حدود تقريبية للكلمة (مسافات/ترقيم عربي-إنجليزي)
        pattern = r'(?:(?<=^)|(?<=[\s،,:;!\.\(\)\[\]\{\}]))(' + '|'.join(map(re.escape, alts)) + r')(?=(?:$|[\s،,:;!\.\(\)\[\]\{\}]))'
        return re.compile(pattern)

    # ========= من هنا منطق "قبل النقطتين" =========
    def _label_from_colon(self, raw: str):
        """
        يستخرج ما قبل النقطتين من السطر:
        مثال: '- الدجاج : 2 صدر ...' → 'الدجاج'
        نتعامل مع أنواع : المختلفة [ : ： ﹕ ︰ ] ونزيل الشرطة/النقاط والمسافات.
        """
        if not raw:
            return None
        left_right = re.split(r'\s*[：:﹕︰]\s*', raw, maxsplit=1)
        if len(left_right) < 2:
            return None
        left = left_right[0]
        # إزالة الرموز في بداية السطر
        left = re.sub(r'^[\-\–\—•\s]+', '', left)
        # تنعيم بسيط: شيل "ال" من كل كلمة (اختياري)
        left = re.sub(r'(^|\s)ال(?=\S)', r'\1', left)
        left = left.strip(' .،:؛-').strip()
        return left or None

    def extract_ingredients_from_text(self, text: str):
        """
        1) نحاول أولاً استخراج label قبل النقطتين —> هذا يرجع مباشرة.
        2) لو ما فيه نقطتين، نلجأ لاستخراج مطابقات القاموس بالـ regex.
        """
        if not text:
            return []

        # أولوية "ما قبل النقطتين"
        lbl = self._label_from_colon(text)
        if lbl:
            return [lbl]

        # احتياطي: تنظيف خفيف ثم مطابقات القاموس
        t = re.sub(r'[\(\[\{].*?[\)\]\}]', ' ', text)                      # شيل الأقواس
        t = re.sub(r'[0-9٠-٩]+([\/\.][0-9٠-٩]+)?', ' ', t)                 # شيل الأرقام/الكسور
        t = re.sub(r'[¼½¾⅓⅔⅛⅜⅝⅞]', ' ', t)
        t_norm = self.normalize_ar(t.replace('ة', 'ه'))                    # للتطابق فقط
        matches = self.ing_regex.findall(t_norm)
        if not matches:
            return []
        # حوّل للـ canonical الجميل + dedupe مع الحفاظ على الترتيب
        out, seen = [], set()
        for m in matches:
            canon = self.syn2canon.get(m)
            if canon and canon not in seen:
                out.append(canon)
                seen.add(canon)
        return out

    # ========= الشبكات =========
    def get_page_content(self, url):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        try:
            resp = requests.get(url, headers=headers, timeout=10)
            resp.raise_for_status()
            return resp.text
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            return None

    def extract_dish_links(self, html_content):
        soup = BeautifulSoup(html_content, 'html.parser')
        links = soup.select('a[href*="/node/"]')
        dish_links = []
        for link in links:
            href = link.get('href')
            if href and '/node/' in href:
                dish_links.append(href if href.startswith('http') else f"{self.base_url}{href}")
        return list(set(dish_links))

    # ========= تفاصيل الطبق =========
    def extract_dish_details(self, dish_url):
        html = self.get_page_content(dish_url)
        if not html:
            return None

        soup = BeautifulSoup(html, 'html.parser')

        # اسم الطبق
        title_element = soup.find('h1') or soup.find('h2') or soup.find('title')
        dish_name = title_element.get_text().strip() if title_element else "Unknown Dish"

        # الصورة
        image_url = None
        meta_image = soup.find('meta', property='og:image')
        if meta_image:
            image_url = meta_image.get('content', '')
        else:
            img = soup.find('img', {'src': re.compile(r'\.(jpg|jpeg|png|webp)', re.I)})
            if img:
                image_url = img.get('src', '')
                if image_url and not image_url.startswith('http'):
                    image_url = f"{self.base_url}{image_url}"

        # المكوّنات: نجمع فقط labels (قبل النقطتين إن وجدت)
        collected = []

        # 1) schema.org
        for el in soup.select('[itemprop="recipeIngredient"]'):
            hits = self.extract_ingredients_from_text(el.get_text(" ", strip=True))
            if hits:
                collected.extend(hits)

        # 2) حاويات بديلة
        if not collected:
            for container in soup.select('.ingredients, .recipe-ingredients, .mkd-recipe-ingredients'):
                for li in container.select('li'):
                    hits = self.extract_ingredients_from_text(li.get_text(" ", strip=True))
                    if hits:
                        collected.extend(hits)
                if collected:
                    break

        # 3) نمط نصي (مقادير/مكونات)
        if not collected:
            for element in soup.find_all(string=re.compile(r'مقادير|مكونات|المقادير|المكونات', re.IGNORECASE)):
                parent = element.find_parent()
                if not parent:
                    continue
                lis = parent.find_all('li')
                if not lis:
                    continue
                for li in lis:
                    hits = self.extract_ingredients_from_text(li.get_text(" ", strip=True))
                    if hits:
                        collected.extend(hits)
                if collected:
                    break

        # إزالة التكرار مع الحفاظ على الترتيب
        ingredients = list(OrderedDict.fromkeys(collected))

        scrape_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return {
            'dish_name': dish_name,
            'ingredients': ingredients,
            'image_url': image_url,
            'dish_url': dish_url,
            'scrape_date': scrape_date
        }

    # ========= التشغيل العام =========
    def scrape_saudi_dishes(self, max_pages=5):
        print("Starting to scrape Saudi dishes...")
        all_links = []
        for page in range(1, max_pages + 1):
            url = f"{self.saudi_cuisine_url}?page={page}" if page > 1 else self.saudi_cuisine_url
            print(f"Scraping page {page}: {url}")
            html = self.get_page_content(url)
            if not html:
                print(f"Failed to retrieve page {page}")
                continue
            links = self.extract_dish_links(html)
            all_links.extend(links)
            print(f"Found {len(links)} dishes on page {page}")
            time.sleep(1)

        all_links = list(set(all_links))
        print(f"Total unique dishes found: {len(all_links)}")

        for i, link in enumerate(all_links, 1):
            print(f"Scraping dish {i}/{len(all_links)}: {link}")
            data = self.extract_dish_details(link)
            if data and data['ingredients']:
                self.dishes_data.append(data)
            time.sleep(1)

        print(f"Successfully scraped {len(self.dishes_data)} dishes with ingredients")
        return self.dishes_data

    # ========= الحفظ =========
    def save_to_csv(self, filename="saudi_dishes.csv"):
        if not self.dishes_data:
            print("No data to save")
            return
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['dish_name', 'ingredients', 'image_url', 'dish_url', 'scrape_date'])
            writer.writeheader()
            for d in self.dishes_data:
                row = d.copy()
                row['ingredients'] = '|'.join(row['ingredients'])
                writer.writerow(row)
        print(f"Data saved to {filename}")

    def save_to_json(self, filename="saudi_dishes.json"):
        if not self.dishes_data:
            print("No data to save")
            return
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.dishes_data, f, ensure_ascii=False, indent=2)
        print(f"Data saved to {filename}")


# ====== التشغيل المباشر ======
if __name__ == "__main__":
    scraper = SaudiDishScraper()
    dishes = scraper.scrape_saudi_dishes(max_pages=3)
    if dishes:
        scraper.save_to_csv()
        scraper.save_to_json()
        print("\nSample:")
        for i, d in enumerate(dishes[:3]):
            print(f"\nDish {i+1}:")
            print("Name:", d['dish_name'])
            print("Ingredients:", d['ingredients'])
            print("Image:", d['image_url'])
            print("URL:", d['dish_url'])


Starting to scrape Saudi dishes...
Scraping page 1: https://kitchen.sayidaty.net/recipes/index/cuisine/2419
Found 18 dishes on page 1
Scraping page 2: https://kitchen.sayidaty.net/recipes/index/cuisine/2419?page=2
Found 19 dishes on page 2
Scraping page 3: https://kitchen.sayidaty.net/recipes/index/cuisine/2419?page=3
Found 19 dishes on page 3
Total unique dishes found: 54
Scraping dish 1/54: https://kitchen.sayidaty.net/node/36100/الخبز-الأحمر-الحساوي/وصفات-الخبز/وصفات
Scraping dish 2/54: https://kitchen.sayidaty.net/node/36099/الحنيني-السعودي/حلويات/وصفات
Scraping dish 3/54: https://kitchen.sayidaty.net/node/34178/اللقيمات-الذهبية-الهشة-بدون-بيض/حلويات/وصفات
Scraping dish 4/54: https://kitchen.sayidaty.net/node/36120/الدبيازة-حلى-ضيافة-لليوم-الوطني-السعودي/حلويات/وصفات
Scraping dish 5/54: https://kitchen.sayidaty.net/node/36110/المصابيب-السعودية/حلويات/وصفات
Scraping dish 6/54: https://kitchen.sayidaty.net/node/36118/قهوة-اللوز-الحجازية-التقليدية/مشروبات-وعصائر/وصفات
Scraping dish 7/